# Set OPENAI API key in Environment Variables and importing Liberaries

In [4]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [5]:
from langchain.chat_models import ChatOpenAI
import os
import traceback



In [6]:
chat = ChatOpenAI(temperature=0.0)
chat

ChatOpenAI(verbose=False, callbacks=None, callback_manager=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.0, model_kwargs={}, openai_api_key=None, openai_api_base=None, openai_organization=None, request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None)

In [7]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.prompts import ChatPromptTemplate


## Defining output format structure using StructuredOutputParser

In [8]:
# Define response schemas for the output of the chatbot, specifying the expected format of each field
type_schema = ResponseSchema(name="code_type", description="type of the code whether it's a function or a script")
old_code_schema = ResponseSchema(name="old_code", description="the original code")
new_code_schema = ResponseSchema(name="new_code", description="the fixed code")
explaination_schema = ResponseSchema(name="explain", description="explain changes")
response_schemas =[type_schema, old_code_schema, new_code_schema, explaination_schema]

# Create a structured output parser using the response schemas
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [9]:
# Get the format instructions for how to format a message to send to the chatbot
format_instructions = output_parser.get_format_instructions()

# Print the format instructions for reference
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"code_type": string  // type of the code whether it's a function or a script
	"old_code": string  // the original code
	"new_code": string  // the fixed code
	"explain": string  // explain changes
}
```


# Defining Prompt Template

In [10]:
# Define a template string for the message to send to the chatbot, using placeholders for the code, error, test cases, and format instructions
template_string = """fix the code \
that is delimited by triple backticks \
code: ```{code}```
that has the following error
{error}
to pass the following test cases
{test_cases_str}

{format_instructions}
"""

# Create a chat prompt template using the template string
prompt_template = ChatPromptTemplate.from_template(template_string)

## Function to check the code for errors and return the traceback of the error

In [11]:
# Define a function to check if a given code and test cases run without errors
def check_code(code, test_cases):
    try:
        # Execute the given code
        exec(code)
        
        # Execute each test case
        for test_case in test_cases:
            exec(test_case)
        
        # If no errors occur, return None to indicate success
        return None
    
    # If an error occurs, catch it and return the error message as a string
    except BaseException as e:
        error = str(traceback.format_exc())
        return error

## Function to fix the code

In [12]:
# Define a function to attempt to fix a given code that fails to pass test cases
def fix_code(code, n, test_cases):
    # Check if the initial code already passes all test cases
    error = check_code(code, test_cases)
    if error is None:
        return code
    
    # If the initial code fails test cases, print the code and error message for reference
    else:
        print(code)
        print()
        print(f"\033[31m{error}\033[0m")
        print("##################################################################")
    
    # Attempt to fix the code
    print("Fixing\n")
    for i in range(n):
        print(f"Trial #{i}")
        
        # Format a message to send to a chatbot, asking for help fixing the code
        code_messages = prompt_template.format_messages(
                            code=code,
                            error=error,
                            test_cases_str="\n".join(test_cases),
                            format_instructions=format_instructions)
        
        # Send the message to the chatbot and parse the response
        response = chat(code_messages)
        output_dict = output_parser.parse(response.content)
        
        # Get the new code from the response and check if it passes all test cases
        code = output_dict['new_code']
        error = check_code(code, test_cases)
        
        # If the new code passes all test cases, return the fixed code and print an explanation
        if error is None:
            print("Explanation: ")
            print(output_dict['explain'])
            print("\033[32mSuccess\033[0m")
            return code
        
        # If the new code still fails test cases, print an explanation and continue to the next trial
        else:
            print("Explanation: ")
            print(output_dict['explain'])
            print("\033[31mFailure\033[0m")
            print("Failure")
            print(f"\033[31m{error}\033[0m")
            print("=="*20)
    
    # If no fixed code is found after n trials, return -1 to indicate failure
    return -1

In [14]:
code = """def can_rearrange(s1, s2):
    # If the strings are identical, then s1 can be rearranged to be s2
    s1=s
    if s1 == s2:
        return True
    else:
        return False"""
error = ""

test_cases=["assert can_rearrange('race', 'care')==True",
           "assert can_rearrange('hello', 'world')==False",
           "assert can_rearrange('listen', 'silent')==True"]

fixed_code = fix_code(code, 5, test_cases)
print(f"\033[32m{fixed_code}\033[0m")

def can_rearrange(s1, s2):
    # If the strings are identical, then s1 can be rearranged to be s2
    s1=s
    if s1 == s2:
        return True
    else:
        return False

Traceback (most recent call last):
  File "/tmp/ipykernel_84/3013963956.py", line 9, in check_code
    exec(test_case)
  File "<string>", line 1, in <module>
  File "<string>", line 3, in can_rearrange
NameError: name 's' is not defined

##################################################################
Fixing

Trial #0
Explanation: 
The variable 's' was not defined, so it was replaced with 's1'. Additionally, the function now sorts both strings and compares them to check if they are rearrangements of each other.
Success
def can_rearrange(s1, s2):
    # If the strings are identical, then s1 can be rearranged to be s2
    if sorted(s1) == sorted(s2):
        return True
    else:
        return False


In [13]:
code = """def can_rearrange(s1, s2):
    # If the strings are identical, then s1 can be rearranged to be s2
    s1=s
    if s1 == s2:
        return True
    else:
        return False"""
error = ""

#the test cases have cases where the function should account for letters case insensitivity like "Race" and "care"
test_cases=["assert can_rearrange('Race', 'care')==True",
           "assert can_rearrange('hello', 'world')==False",
           "assert can_rearrange('lIsten', 'silent')==True"]

fixed_code = fix_code(code, 5, test_cases)
print(f"\033[32m{fixed_code}\033[0m")

def can_rearrange(s1, s2):
    # If the strings are identical, then s1 can be rearranged to be s2
    s1=s
    if s1 == s2:
        return True
    else:
        return False

Traceback (most recent call last):
  File "/tmp/ipykernel_84/3013963956.py", line 9, in check_code
    exec(test_case)
  File "<string>", line 1, in <module>
  File "<string>", line 3, in can_rearrange
NameError: name 's' is not defined

##################################################################
Fixing

Trial #0
Explanation: 
Fixed the NameError by removing the line that assigns s1 to s. Also, changed the logic to sort the strings and compare them ignoring case.
Success
def can_rearrange(s1, s2):
    # If the strings are identical, then s1 can be rearranged to be s2
    if sorted(s1.lower()) == sorted(s2.lower()):
        return True
    else:
        return False
